In [1]:
import keras
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import np_utils
from bayes_opt import BayesianOptimization

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('https://github.com/ozt-ca/tjo.hatenablog.samples/raw/master/r_samples/public_lib/jp/exp_uci_datasets/online_news_popularity/ONP_train.csv')
df_test = pd.read_csv('https://github.com/ozt-ca/tjo.hatenablog.samples/raw/master/r_samples/public_lib/jp/exp_uci_datasets/online_news_popularity/ONP_test.csv')

x_train = df_train.iloc[:, :58]
y_train = df_train["shares"]
x_test = df_test.iloc[:, :58]
y_test = df_test["shares"]

merged_train = pd.concat([x_train, x_test])

scaler = StandardScaler()
merged_train.iloc[:, 0:10] = scaler.fit_transform(merged_train.iloc[:, 0:10])
merged_train.iloc[:, 17:28] = scaler.fit_transform(merged_train.iloc[:, 17:28])
merged_train.iloc[:, 37:] = scaler.fit_transform(merged_train.iloc[:, 37:])

x_train = merged_train.iloc[:len(df_train), :]
x_test = merged_train.iloc[len(df_train):, :]

In [3]:
df_train = shuffle(df_train)
idx = int(np.round(len(df_train) * 0.9, 0))
x_ptrain = df_train.iloc[:idx, :58]
y_ptrain = df_train.iloc[:idx, 58]
x_ptest = df_train.iloc[idx:, :58]
y_ptest = df_train.iloc[idx:, 58]

In [4]:
def get_model(unit1, unit2, unit3):
    model = Sequential()
    model.add(Dense(int(unit1), input_dim = 58))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(int(unit2)))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(int(unit3)))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('linear'))
    return model

def fit_with(unit1, unit2, unit3, lr, num_epoch):
    model = get_model(unit1, unit2, unit3)
    sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss = 'mean_squared_error', optimizer = sgd)
    model.fit(x_ptrain, y_ptrain, epochs = int(num_epoch), batch_size = 200, verbose = 0)
    pred = model.predict(x_ptest)
    rmse = np.sqrt(mean_squared_error(pred, y_ptest))
    return -rmse

pbounds = {'unit1': (200, 300), 'unit2': (120, 160), 'unit3': (60, 80),
           'lr': (1e-3, 1e-2), 'num_epoch': (50, 100)}

optimizer = BayesianOptimization(
    f = fit_with,
    pbounds = pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=1, n_iter=1,)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.res['max']['max_params'])


Initialization
-------------------------------------------------------------------------------------------
 Step |   Time |      Value |        lr |   num_epoch |     unit1 |     unit2 |     unit3 | 
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
    1 | 00m31s |   -0.90885 |    0.0023 |     65.1166 |  200.0114 |  148.8130 |   68.3404 | 
Bayesian Optimization
-------------------------------------------------------------------------------------------
 Step |   Time |      Value |        lr |   num_epoch |     unit1 |     unit2 |     unit3 | 
    2 | 00m40s |   -0.91143 |    0.0049 |     75.2108 |  282.8395 |  142.3345 |   63.7524 | 
Iteration 0: 
	max
Iteration 1: 
	all
{'unit3': 68.34044009405147, 'unit2': 148.81297973768633, 'unit1': 200.01143748173448, 'num_epoch': 65.116628631592, 'lr': 0.002320803

In [10]:
model = Sequential()
model.add(Dense(int(optimizer.res['max']['max_params']['unit1']), input_dim = 58))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(int(optimizer.res['max']['max_params']['unit2'])))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(int(optimizer.res['max']['max_params']['unit3'])))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('linear'))

sgd = SGD(lr=optimizer.res['max']['max_params']['lr'], decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = 'mean_squared_error', optimizer = sgd)
cb = keras.callbacks.EarlyStopping(monitor='loss', patience=10, mode='min')
model.fit(x_train, y_train, nb_epoch = 100, batch_size = 200, verbose = 1, callbacks = [cb])
pred = model.predict(x_test)
rmse = np.sqrt(mean_squared_error(pred, y_test))
print "RMSE:", rmse

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


Epoch 1/100
34644/34644 [==============================] - 1s 28us/step - loss: 1.6151
Epoch 2/100
34644/34644 [==============================] - 1s 15us/step - loss: 0.8712
Epoch 3/100
34644/34644 [==============================] - 1s 15us/step - loss: 0.8669
Epoch 4/100
34644/34644 [==============================] - 1s 16us/step - loss: 0.8696
Epoch 5/100
34644/34644 [==============================] - 1s 16us/step - loss: 0.8649
Epoch 6/100
34644/34644 [==============================] - 1s 16us/step - loss: 0.8656
Epoch 7/100
34644/34644 [==============================] - 1s 17us/step - loss: 0.8640
Epoch 8/100
34644/34644 [==============================] - 1s 15us/step - loss: 0.8652
Epoch 9/100
34644/34644 [==============================] - 1s 16us/step - loss: 0.8631
Epoch 10/100
34644/34644 [==============================] - 1s 16us/step - loss: 0.8641
Epoch 11/100
34644/34644 [==============================] - 1s 16us/step - loss: 0.8639
Epoch 12/100
34644/34644 [===============